In [7]:
from collections import defaultdict

def two_break_dist(P, Q):

    graph = defaultdict(list)

    for perm_cycle in P + Q :

        L = len(perm_cycle)

        for i in xrange(len(perm_cycle)) :
            graph[perm_cycle[i]].append(-1 * perm_cycle[(i + 1) % L])
            graph[-1 * perm_cycle[(i + 1) % L]].append(perm_cycle[i])

    component_count = 0
    remaining = set(graph.keys())

    while len(remaining) > 0 :
        component_count += 1
        queue = [remaining.pop()]

        while queue :
            current = queue.pop(0)
            queue += filter(lambda node: node in remaining, graph.get(current, []))
            remaining -= set(queue)

    return sum(map(len,P)) - component_count

input_file = open("untitled.txt", 'r')
P, Q = [line.strip().lstrip('(').rstrip(')').split(')(') for line in input_file.readlines()]
P = [map(int, block.split()) for block in P]
Q = [map(int, block.split()) for block in Q]

dist = two_break_dist(P, Q)

print str(dist)

2


In [16]:
import sys
import random

input_file = open("untitled.txt", 'r')
p = map(int, input_file.readline().replace('\n', '')[1 : -1].split(' '))
q = map(int, input_file.readline().replace('\n', '')[1 : -1].split(' '))

def ChromosomeToCycle(chromosome) :
    
    nodes = []
    
    for i in chromosome :
        if(i > 0) :
            nodes.append(2 * i - 1)
            nodes.append(2 * i)
        else :
            nodes.append(-2 * i)
            nodes.append(-2 * i - 1)
            
    return nodes

def CycleToChromosome(nodes) :
    
    chromosome = []
    
    for j in range(1, len(nodes) / 2 + 1) :
        if nodes[2 * j - 2] < nodes[2 * j - 1] : chromosome.append(nodes[2 * j - 1]/2)
        else : chromosome.append(-nodes[2 * j - 2] / 2)
    return chromosome

def ColoredEdges(p) :
    
    edges = []
    
    if(type(p[0]) == int) :
        chromosome = p
        chromosome.append(chromosome[0])
        nodes = ChromosomeToCycle(chromosome)
        for j in range(1, len(chromosome)) : edges.append([nodes[2 * j - 1],nodes[2 * j]])
        chromosome.pop()
        
        return edges
    
    else :
        for chromosome in p :
            temp = chromosome[0]
            chromosome.append(temp)
            nodes = ChromosomeToCycle(chromosome)
            for j in range(1, len(chromosome)) : edges.append([nodes[2 * j - 1],nodes[2 * j]])
            chromosome.pop()
            
        return edges
    
def get_blacked_edges(col_edges) :
    
    blacked_edges = []
    
    for node in col_edges :
        temp = []
        for num in node :
            if(num % 2 == 0) : temp.append(num / 2)
            else : temp.append(-(num + 1) / 2)
        blacked_edges.append(temp)
        
    return blacked_edges

def GraphToGenome(col_edges) :
    
    blacked_edges = get_blacked_edges(col_edges)
    chromosome_list = []
    
    while blacked_edges != [] :
        
        start, end = blacked_edges[0]
        blacked_edges.remove(blacked_edges[0])
        
        loop_end = -1 * start
        chromosome = [start]
        
        while end != loop_end :
            for i in range(len(blacked_edges)) :
                if(blacked_edges[i][0] == -1 * end) :
                    start,end = blacked_edges[i]
                    blacked_edges.remove(blacked_edges[i])
                    break
                    
                elif(blacked_edges[i][1] == -1 * end) :
                    blacked_edges[i] = blacked_edges[i][::-1] 
                    start,end = blacked_edges[i]
                    blacked_edges.remove(blacked_edges[i])
                    break
        
            chromosome.append(start)
        chromosome_list.append(chromosome)
        
    return chromosome_list

def two_breakongenomegraph(col_edges, i, j, k, l) :
    
    if [i,j] in col_edges : col_edges.remove([i, j])
    else : col_edges.remove([j,i])
        
    if [k,l] in col_edges : col_edges.remove([k, l])
    else : col_edges.remove([l, k])
        
    col_edges.append([i, k])
    col_edges.append([j, l])
    
    return col_edges

def two_breakongenome(p, i, j, k, l) :
    
    colored_edges = ColoredEdges(p)
    genomegraph = two_breakongenomegraph(colored_edges, i, j, k, l)
    ans = GraphToGenome(genomegraph)
    
    return ans

def shortestrearrangement(p,q) :

    print '(' + ' '.join('+' + str(i) if i > 0 else str(i) for i in p) + ')'
    
    red_edges = ColoredEdges(p)
    blue_edges = ColoredEdges(q)
    
    breakpointgraph = red_edges + blue_edges
    
    while True :
        
        red_edges = ColoredEdges(p)
        
        for i in range(len(red_edges)) :
            if red_edges[i][::-1] in blue_edges : red_edges[i] = red_edges[i][::-1]
        if(sorted(red_edges) == sorted(blue_edges)) : break
            
        temp_list = []
        
        for blue_edge in blue_edges :
            if blue_edge not in red_edges : temp_list.append(blue_edge)
                
        node = random.choice(temp_list)
        
        for red_edge in red_edges :
            if node[0] in red_edge : to_edge = red_edge
            elif node[1] in red_edge : from_edge = red_edge
                
        if((to_edge[1]==node[0]) & (from_edge[0]==node[1])) :
            red_edges.remove(from_edge)
            red_edges.remove(to_edge)
            red_edges.append([to_edge[1],from_edge[0]])
            red_edges.append([to_edge[0],from_edge[1]])

        elif((to_edge[0]==node[0]) & (from_edge[0]==node[1])) :
            red_edges.remove(from_edge)
            red_edges.remove(to_edge)
            to_edge = to_edge[::-1]
            red_edges.append([to_edge[1],from_edge[0]])
            red_edges.append([to_edge[0],from_edge[1]])
    
        elif((to_edge[1]==node[0]) & (from_edge[1]==node[1])) :
            red_edges.remove(from_edge)
            red_edges.remove(to_edge)
            from_edge = from_edge[::-1]
            red_edges.append([to_edge[1],from_edge[0]])
            red_edges.append([to_edge[0],from_edge[1]])
            
        elif((to_edge[0]==node[0]) & (from_edge[1]==node[1])) :
            red_edges.remove(from_edge)
            red_edges.remove(to_edge)
            from_edge = from_edge[::-1]
            to_edge = to_edge[::-1]
            red_edges.append([to_edge[1],from_edge[0]])
            red_edges.append([to_edge[0],from_edge[1]])
            
        breakpointgraph = red_edges + blue_edges
        p = two_breakongenome(p, to_edge[0], to_edge[1], from_edge[1], from_edge[0])
        
        str_p = ''
        
        for res in p : str_p += '(' + ' '.join('+' + str(i) if i > 0 else str(i) for i in res) + ')'
        print str_p
        
        if len(p) == 1 : p = p[0]

    return None

shortestrearrangement(p, q)

ValueError: invalid literal for int() with base 10: '+4)(+5'

In [4]:
import sys
from collections import defaultdict

input_file = open("untitled.txt", 'r')

k = int(input_file.readline().replace('\n', ''))
p = input_file.readline().replace('\n', '')
q = input_file.readline().replace('\n', '')

def rev_comp(sequence) :

    rev_comp = ''
    
    for nucleotide in sequence :
        if(nucleotide == 'A') : rev_comp = rev_comp + 'T'
        elif(nucleotide == 'T') : rev_comp = rev_comp + 'A'
        elif(nucleotide == 'G') : rev_comp = rev_comp + 'C'
        elif(nucleotide == 'C') : rev_comp = rev_comp + 'G'
            
    rev_comp = rev_comp[::-1]
    return rev_comp

def sharedkmer(dna1,dna2) :

    dna1_dict = defaultdict(list)
    for i in xrange(len(dna1) - k + 1) : dna1_dict[dna1[i : i + k]].append(i)
        
    shared_kmer_indices = set()
    
    for j in xrange(len(dna2) - k + 1) :
        shared_kmer_indices |= set(map(lambda x : (x, j), dna1_dict[dna2[j : j + k]]))
        shared_kmer_indices |= set(map(lambda x : (x, j), dna1_dict[rev_comp(dna2[j : j + k])]))
        
    return shared_kmer_indices

ans = sharedkmer(p, q)
for res in ans : print res

ValueError: invalid literal for int() with base 10: '(+1 +2 +3 +4)(+5 -9 -8 -7 +6)'